In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from google.colab import files
from sklearn.model_selection import train_test_split
import warnings; warnings.simplefilter('ignore')

In [ ]:
def import_dataset():
  for _ in range(4):
    uploaded = files.upload()
  user_set = pd.read_csv("user_set.csv")
  job_set = pd.read_csv("job_set_cleaned.csv")
  dataset = pd.read_csv("dataset.csv")
  work_history = pd.read_csv("work_history.csv")
  return user_set, job_set, dataset, work_history

In [ ]:
# uploaded = files.upload()
# dataset = pd.read_csv("dataset.csv")

In [ ]:
user_set, job_set, dataset, work_history = import_dataset()

Saving dataset.csv to dataset.csv


Saving work_history.csv to work_history.csv


Saving user_set.csv to user_set.csv


Saving job_set_cleaned.csv to job_set_cleaned.csv


## TF-IDF Vectors for text representation

In [ ]:
job_set = job_set.fillna(" ")
job_set["word"] = job_set.title + job_set.Description 
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=5, max_features=100, stop_words='english')
tfidf_matrix = tf.fit_transform(job_set['word'])

In [ ]:
dataset.columns

Index(['UserID', 'JobID', 'label'], dtype='object')

In [ ]:
user_set.columns

Index(['Unnamed: 0', 'uid', 'country', 'language', 'birthdate', 'joined'], dtype='object')

In [ ]:
user_id = dataset.UserID.unique()
user_set = user_set[user_set.uid.isin(user_id)]
user_set.reset_index(drop=True, inplace=True)

In [ ]:
word_history_tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, max_features=50, stop_words='english')
word_history_tf_matrix = word_history_tf.fit_transform(work_history.groupby("user_id").title.sum().values)

In [ ]:
work_history.groupby("user_id").title.sum()

user_id
1.675922e+09    Object recognition & detectionObject recogniti...
1.709755e+09    [Toloka] English Grammar Test[Toloka] English ...
1.718146e+09    Check the similarity between two productsCheck...
1.718416e+09    Check the similarity between two productsCheck...
Name: title, dtype: object

In [ ]:
print(word_history_tf_matrix[0,:])

  (0, 6)	0.12607512957032954
  (0, 26)	0.12607512957032954
  (0, 35)	0.12607512957032954
  (0, 34)	0.08344783645915785
  (0, 28)	0.08344783645915785
  (0, 39)	0.08344783645915785
  (0, 9)	0.08344783645915785
  (0, 12)	0.08344783645915785
  (0, 8)	0.12517175468873676
  (0, 13)	0.08344783645915785
  (0, 2)	0.12517175468873676
  (0, 20)	0.12517175468873676
  (0, 24)	0.08344783645915785
  (0, 18)	0.12517175468873676
  (0, 30)	0.1668956729183157
  (0, 40)	0.08344783645915785
  (0, 15)	0.15310302455421257
  (0, 10)	0.12607512957032954
  (0, 5)	0.12607512957032954
  (0, 25)	0.12607512957032954
  (0, 37)	0.1668956729183157
  (0, 33)	0.1668956729183157
  (0, 27)	0.29206742760705245
  (0, 38)	0.29206742760705245
  (0, 16)	0.12607512957032954
  (0, 22)	0.2521502591406591
  (0, 7)	0.2086195911478946
  (0, 0)	0.041723918229578924
  (0, 4)	0.041723918229578924
  (0, 43)	0.08344783645915785
  (0, 21)	0.37551526406621033
  (0, 29)	0.12517175468873676
  (0, 36)	0.2521502591406591
  (0, 46)	0.0417239182

## Deal with User Set and Job Set

In [ ]:
user_set = user_set.drop(columns=["country"])

In [ ]:
user_set.head()

,Unnamed: 0,uid,language,birthdate,joined
0,0,1316040758,"AR,EN,",1984-03-11T00:00:00.000Z,2021-01-13T00:00:00.000Z
1,1,1429235843,"EN,KI,SW,",1993-03-01T00:00:00.000Z,2021-05-27T00:00:00.000Z
2,2,1430094402,"DE,EN,ES,ID,PT,RU,",1989-09-24T00:00:00.000Z,2021-05-29T00:00:00.000Z
3,3,1458049517,"DE,EN,FR,RU,TR,",1996-12-16T00:00:00.000Z,2021-07-27T00:00:00.000Z
4,4,1490584733,"EN,ID,",1982-03-13T00:00:00.000Z,2021-09-29T00:00:00.000Z


## create age and years worked feature

In [ ]:
# # This function converts given date to age

# import pandas as pd
# from datetime import datetime, date
  

# def age(born):
#     # born = pd.to_datetime(born)
#     # born = born.dt.strftime('%m/%d/%Y')
#     born = datetime.strptime(born, "%Y/%m/%d, %H:%M:%S").date()
    
#     # "%d/%m/%Y"
#     today = date.today()
#     return today.year - born.year - ((today.month, 
#                                       today.day) < (born.month, 
#                                                     born.day))
  
# user_set['Age'] = user_set['birthdate'].apply(age)
# user_set['Years_Exp'] = user_set['joined'].apply(age)
# display(user_set)

In [ ]:
def age(birthdate):
  today = date.today()
  age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
  return age

In [ ]:
from datetime import date
user_set['birthdate'] = user_set['birthdate'].astype(str).str[:10]
user_set['birthdate'] = pd.to_datetime(user_set['birthdate'], errors='coerce')
age_list = []
for i in range(len(user_set['birthdate'])):
  ages = age(user_set['birthdate'][i])
  age_list.append(ages)
user_set['age'] = age_list

In [ ]:
from datetime import date
user_set['joined'] = user_set['joined'].astype(str).str[:10]
user_set['joined'] = pd.to_datetime(user_set['joined'], errors='coerce')
age_list = []
for i in range(len(user_set['joined'])):
  ages = age(user_set['joined'][i])
  age_list.append(ages)
user_set['Years_of_exp'] = age_list

In [ ]:
user_set.head()

,Unnamed: 0,uid,language,birthdate,joined,age,Years_of_exp
0,0,1316040758,"AR,EN,",1984-03-11,2021-01-13,39,2
1,1,1429235843,"EN,KI,SW,",1993-03-01,2021-05-27,30,1
2,2,1430094402,"DE,EN,ES,ID,PT,RU,",1989-09-24,2021-05-29,33,1
3,3,1458049517,"DE,EN,FR,RU,TR,",1996-12-16,2021-07-27,26,1
4,4,1490584733,"EN,ID,",1982-03-13,2021-09-29,41,1


In [ ]:
job_set.head()

,Unnamed: 0,project_id,pool_id,title,description,pool_startedAt,hasInstructions,mayContainAdultContent,requesterID,requesterTrusted,lang,grade,averageAcceptanceTimeSec,moneyAvgHourly,moneyMed,moneyTop10,moneyMax3,reward,Description,word
0,1,2338,36616149,Ekran görüntüsünde yetişkin içerik var mı?,"Bu, ""Ekran görüntüsünde yetişkin içerik var mı...",2022-11-29T12:19:54.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,,4.52,,0.0,0.0,0.09,1.19,0.0,"Bu, ""Ekran görüntüsünde yetişkin içerik var mı...","Ekran görüntüsünde yetişkin içerik var mı?Bu, ..."
1,2,2338,36760808,Ekran görüntüsünde yetişkin içerik var mı?,Değerlendirilen görsele en uygun kategoriyi se...,2022-12-08T22:27:28.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,,4.54,,0.0,0.0,0.29,1.69,0.0,Değerlendirilen görsele en uygun kategoriyi se...,Ekran görüntüsünde yetişkin içerik var mı?Değe...
2,3,2338,36774752,Ekran görüntüsünde yetişkin içerik var mı?,Değerlendirilen görsele en uygun kategoriyi se...,2022-12-09T21:50:47.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,,4.51,,0.0,0.0,0.09,1.19,0.0,Değerlendirilen görsele en uygun kategoriyi se...,Ekran görüntüsünde yetişkin içerik var mı?Değe...
3,4,6795,21626385,Тональность сообщений из социальных сетей,Разметка сообщений по эмоциональной окраске.,2022-10-23T22:24:44.000Z,1,1,81029b6e4b04badacad20d15ff3998e0,1,,4.89,,2.4,0.04,0.14,,0.0,Разметка сообщений по эмоциональной окраске.,Тональность сообщений из социальных сетей Разм...
4,5,7053,32956274,Яндекс Маркет. Редиректы. Категория. Точность,"Вам предлагается оценить, идеально ли предложе...",2022-04-15T05:59:48.000Z,1,1,7d935a4f05dc1f1f5ec7452feba85b5c,1,,4.92,,0.75,0.04,0.04,1.74,0.0,"Вам предлагается оценить, идеально ли предложе...",Яндекс Маркет. Редиректы. Категория. ТочностьВ...


add binary labels into the dataset, indicating that whether the user and job are in the same language

In [ ]:
pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=012df487e65acfcb2421969f0db01e66287d1d1519e132e380896415fed8cc25
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [ ]:
from langdetect import detect
job_set['language'] = job_set['title'].apply(detect)

In [ ]:
def split_lang(a):
  my_list = a.split(",")
  my_list = [element.lower() for element in my_list]
  new_list = my_list[:-1]
  return new_list
  
user_set['lang'] = user_set['language'].apply(split_lang)

In [ ]:
user_set.head()

,Unnamed: 0,uid,language,birthdate,joined,age,Years_of_exp,lang
0,0,1316040758,"AR,EN,",1984-03-11,2021-01-13,39,2,"[ar, en]"
1,1,1429235843,"EN,KI,SW,",1993-03-01,2021-05-27,30,1,"[en, ki, sw]"
2,2,1430094402,"DE,EN,ES,ID,PT,RU,",1989-09-24,2021-05-29,33,1,"[de, en, es, id, pt, ru]"
3,3,1458049517,"DE,EN,FR,RU,TR,",1996-12-16,2021-07-27,26,1,"[de, en, fr, ru, tr]"
4,4,1490584733,"EN,ID,",1982-03-13,2021-09-29,41,1,"[en, id]"


In [ ]:
job_set.head()

,Unnamed: 0,project_id,pool_id,title,description,pool_startedAt,hasInstructions,mayContainAdultContent,requesterID,requesterTrusted,...,grade,averageAcceptanceTimeSec,moneyAvgHourly,moneyMed,moneyTop10,moneyMax3,reward,Description,word,language
0,1,2338,36616149,Ekran görüntüsünde yetişkin içerik var mı?,"Bu, ""Ekran görüntüsünde yetişkin içerik var mı...",2022-11-29T12:19:54.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,...,4.52,,0.0,0.0,0.09,1.19,0.0,"Bu, ""Ekran görüntüsünde yetişkin içerik var mı...","Ekran görüntüsünde yetişkin içerik var mı?Bu, ...",tr
1,2,2338,36760808,Ekran görüntüsünde yetişkin içerik var mı?,Değerlendirilen görsele en uygun kategoriyi se...,2022-12-08T22:27:28.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,...,4.54,,0.0,0.0,0.29,1.69,0.0,Değerlendirilen görsele en uygun kategoriyi se...,Ekran görüntüsünde yetişkin içerik var mı?Değe...,tr
2,3,2338,36774752,Ekran görüntüsünde yetişkin içerik var mı?,Değerlendirilen görsele en uygun kategoriyi se...,2022-12-09T21:50:47.000Z,1,1,fa517ea3113e489138d249ff9844f572,1,...,4.51,,0.0,0.0,0.09,1.19,0.0,Değerlendirilen görsele en uygun kategoriyi se...,Ekran görüntüsünde yetişkin içerik var mı?Değe...,tr
3,4,6795,21626385,Тональность сообщений из социальных сетей,Разметка сообщений по эмоциональной окраске.,2022-10-23T22:24:44.000Z,1,1,81029b6e4b04badacad20d15ff3998e0,1,...,4.89,,2.4,0.04,0.14,,0.0,Разметка сообщений по эмоциональной окраске.,Тональность сообщений из социальных сетей Разм...,ru
4,5,7053,32956274,Яндекс Маркет. Редиректы. Категория. Точность,"Вам предлагается оценить, идеально ли предложе...",2022-04-15T05:59:48.000Z,1,1,7d935a4f05dc1f1f5ec7452feba85b5c,1,...,4.92,,0.75,0.04,0.04,1.74,0.0,"Вам предлагается оценить, идеально ли предложе...",Яндекс Маркет. Редиректы. Категория. ТочностьВ...,ru


In [ ]:
dataset.columns

Index(['UserID', 'JobID', 'label'], dtype='object')

In [ ]:
user_set.columns

Index(['Unnamed: 0', 'uid', 'language', 'birthdate', 'joined', 'age',
       'Years_of_exp', 'lang'],
      dtype='object')

In [ ]:
user_set['uid']

0     1316040758
1     1429235843
2     1430094402
3     1458049517
4     1490584733
5     1648608319
6     1674749132
7     1675921760
8     1684019585
9     1703905621
10    1706440161
11    1709755371
12    1710096477
13    1713698815
14    1714073899
15    1716528528
16    1718146027
17    1718416044
18    1725201772
Name: uid, dtype: int64

In [ ]:
job_set.columns

Index(['Unnamed: 0', 'project_id', 'pool_id', 'title', 'description',
       'pool_startedAt', 'hasInstructions', 'mayContainAdultContent',
       'requesterID', 'requesterTrusted', 'lang', 'grade',
       'averageAcceptanceTimeSec', 'moneyAvgHourly', 'moneyMed', 'moneyTop10',
       'moneyMax3', 'reward', 'Description', 'word', 'language'],
      dtype='object')

In [ ]:
x = user_set[user_set.uid==1316040758]["lang"]
print(type(x))
x = x.tolist()
print(type(x))

<class 'pandas.core.series.Series'>
<class 'list'>


In [ ]:
# temp = sorted(dict(dataset.UserID.value_counts()).items(), key=lambda x: x[1], reverse=True)
# exclude_user_id = [i[0] for i in temp if i [1]>=10]
# len(exclude_user_id)

In [ ]:
# dataset = dataset[~dataset.UserID.isin(exclude_user_id)]

In [ ]:
language = []
groups = dataset.groupby("UserID")
for idx, group in tqdm(groups):
    user_language = user_set[user_set.uid==idx]["lang"]
    user_language = user_language.tolist()
    job_id_list = group.JobID.values
    job_language = job_set[job_set.project_id.isin(job_id_list)]["language"].values
    # print(job_language)
    language.extend([0 if i!=user_language else 1 for i in job_language])
    # print(len(language))
# print(len(dataset))
dataset["Language"] = language[:2106]

100%|██████████| 19/19 [00:00<00:00, 746.65it/s]


In [ ]:
print(dataset.shape)
print(job_set.shape)
print(user_set.shape)

(2106, 4)
(561, 21)
(19, 8)


In [ ]:
user_set.to_csv("user_set_cleaned.csv", index=False)
dataset.to_csv("dataset_cleaned.csv", index=False)

## Building the training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
train_user, test_user = train_test_split(user_set, test_size=0.200, random_state=42)
# train_data, test_data = train_test_split(dataset, test_size=0.200, random_state=42)

In [ ]:
train_user = train_user.uid.values
test_user = test_user.uid.values
train_data = dataset[dataset.UserID.isin(train_user)]
test_data = dataset[dataset.UserID.isin(test_user)]

In [ ]:
# train_user = user_set[user_set.Split=="Train"].UserID.values
# test_user = user_set[user_set.Split=="Test"].UserID.values
# train_data = dataset[dataset.UserID.isin(train_user)]
# test_data = dataset[dataset.UserID.isin(test_user)]

In [ ]:
user_set.columns

Index(['Unnamed: 0', 'uid', 'language', 'birthdate', 'joined', 'age',
       'Years_of_exp', 'lang'],
      dtype='object')

In [ ]:
dataset.head()

,UserID,JobID,label,Language
0,1316040758,49390,1,0
1,1316040758,54170,1,0
2,1316040758,73686,1,0
3,1316040758,80759,1,0
4,1316040758,81112,1,0


In [ ]:
user = new[new.uid==1458049517  ]
u_idx = user.index.values[0]
print("USER:", user)
print("U_IDX", u_idx)
print("USER_VALUES", user.values)
print("WORK_HISTORY", word_history_tf_matrix[u_idx,:])
print("LEN_WORK_HIST", word_history_tf_matrix[u_idx,:].shape)

USER:    Unnamed: 0         uid         language  birthdate     joined  age  \
3           3  1458049517  DE,EN,FR,RU,TR, 1996-12-16 2021-07-27   26   

   Years_of_exp                  lang  
3             1  [de, en, fr, ru, tr]  
U_IDX 3
USER_VALUES [[3 1458049517 'DE,EN,FR,RU,TR,' Timestamp('1996-12-16 00:00:00')
  Timestamp('2021-07-27 00:00:00') 26 1
  list(['de', 'en', 'fr', 'ru', 'tr'])]]
WORK_HISTORY   (0, 32)	0.22369638945257625
  (0, 42)	0.22369638945257625
  (0, 14)	0.09210318139293894
  (0, 31)	0.22369638945257625
  (0, 41)	0.37282731575429373
  (0, 34)	0.12192430409454671
  (0, 28)	0.12192430409454671
  (0, 39)	0.12192430409454671
  (0, 9)	0.12192430409454671
  (0, 12)	0.06096215204727336
  (0, 8)	0.12192430409454671
  (0, 13)	0.12192430409454671
  (0, 2)	0.18288645614182006
  (0, 20)	0.18288645614182006
  (0, 24)	0.12192430409454671
  (0, 18)	0.18288645614182006
  (0, 30)	0.06096215204727336
  (0, 40)	0.12192430409454671
  (0, 37)	0.24384860818909343
  (0, 33)	0.12192430

In [ ]:
new = user_set

In [ ]:
new.head()

,Unnamed: 0,uid,language,birthdate,joined,age,Years_of_exp,lang
0,0,1316040758,"AR,EN,",1984-03-11,2021-01-13,39,2,"[ar, en]"
1,1,1429235843,"EN,KI,SW,",1993-03-01,2021-05-27,30,1,"[en, ki, sw]"
2,2,1430094402,"DE,EN,ES,ID,PT,RU,",1989-09-24,2021-05-29,33,1,"[de, en, es, id, pt, ru]"
3,3,1458049517,"DE,EN,FR,RU,TR,",1996-12-16,2021-07-27,26,1,"[de, en, fr, ru, tr]"
5,5,1648608319,"DE,EN,FR,IT,NL,RU,",1993-06-03,2022-06-17,29,0,"[de, en, fr, it, nl, ru]"


In [ ]:
new.drop(new[new['uid'] == 1490584733 ].index, inplace = True)

In [ ]:
groups = train_data.groupby("UserID")
X_train = np.zeros((1,158))
Y_train = []
for u_id, group in tqdm(groups):
    print(u_id, group)
    user = user_set[user_set.uid==u_id]
    u_idx = user.index.values[0]

In [ ]:
word_history_tf_matrix[u_idx,:]

<4x50 sparse matrix of type '<class 'numpy.float64'>'
	with 163 stored elements in Compressed Sparse Row format>

In [ ]:
train_data1 = train_data

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1490584733 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1674749132 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1675921760 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1684019585 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1703905621 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1706440161 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1710096477 ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] ==1713698815  ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1714073899   ].index, inplace = True)

In [ ]:
train_data1.drop(train_data1 [train_data1 ['UserID'] == 1725201772            ].index, inplace = True)

In [ ]:
groups = train_data1.groupby("UserID")
print(len(groups))
X_train = np.zeros((1,158))
Y_train = []
for u_id, group in tqdm(groups):
    print(u_id, group)
    user = user_set[user_set.uid==u_id]
    u_idx = user.index.values[0]
    user_feature = np.concatenate((user.values, word_history_tf_matrix[u_idx,:].toarray()),axis=1)
    # user_feature = np.concatenate((user.values))
    job_id_list = group.JobID.values
    jobs = job_set[job_set.project_id.isin(job_id_list)]
    j_idx = jobs.index.values
    f = []
    for i in j_idx:
        feature = np.concatenate((user_feature, tfidf_matrix[i,:].toarray()), axis=1).reshape(158,).tolist()
        f.append(feature)
    # feature = np.concatenate((group[["language", "language"]].values, np.array(f)),axis=1)
    X_train = np.concatenate((X_train, f), axis=0)
    Y_train.extend(group.label.values.tolist())


In [ ]:
X_train.shape, len(Y_train)

((217, 158), 148)

In [ ]:
test_data1 = test_data

In [ ]:
test_data1.drop(test_data1 [test_data1 ['UserID'] == 1648608319 ].index, inplace = True)

In [ ]:
test_data1.drop(test_data1 [test_data1 ['UserID'] == 1709755371].index, inplace = True)

In [ ]:
test_data1.drop(test_data1 [test_data1 ['UserID'] == 1648608319 ].index, inplace = True)

In [ ]:
test_data1.drop(test_data1 [test_data1 ['UserID'] == 1648608319 ].index, inplace = True)

In [ ]:
test_data1.drop(test_data1 [test_data1 ['UserID'] == 1648608319 ].index, inplace = True)

In [ ]:
groups = test_data1.groupby("UserID")
X_test = np.zeros((1,158))
Y_test = []
for u_id, group in tqdm(groups):
    print(u_id)
    user = user_set[user_set.uid==u_id]
    u_idx = user.index.values[0]
    user_feature = np.concatenate((user.values, word_history_tf_matrix[u_idx,:].toarray()),axis=1)
    job_id_list = group.JobID.values
    jobs = job_set[job_set.project_id.isin(job_id_list)]
    j_idx = jobs.index.values
    f = []
    for i in j_idx:
        feature = np.concatenate((user_feature, tfidf_matrix[i,:].toarray()), axis=1).reshape(158,).tolist()
        f.append(feature)
    # feature = np.concatenate((group[["City","State"]].values, np.array(f)),axis=1)
    X_test = np.concatenate((X_test, f), axis=0)
    Y_test.extend(group.label.values.tolist())

100%|██████████| 2/2 [00:00<00:00, 12.96it/s]

1316040758
1429235843


In [ ]:
X_test.shape, len(Y_test)

((158, 158), 76)

In [ ]:
np.save("X_train.npy",X_train[1:,])
np.save("Y_train.npy",np.array(Y_train))
np.save("X_test.npy",X_test[1:,])
np.save("Y_test.npy",np.array(Y_test))


## Construct Models

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
def show_result(y_true, y_prediction):
    report = classification_report(y_true,y_prediction,digits=4)
    report = report.splitlines()
    columns = ['class'] + report[0].split()
    col_1, col_2, col_3, col_4, col_5 = [], [], [], [], []
    for row in report[1:]:
        if len(row.split()) != 0:
            row = row.split()
            if len(row) < 5:
                col_1.append(row[0])
                col_2.append('')
                col_3.append('')
                col_4.append(row[1])
                col_5.append(row[2])
            elif len(row) > 5:
                col_1.append(row[0] + ' ' + row[1])
                col_2.append(row[2])
                col_3.append(row[3])
                col_4.append(row[4])
                col_5.append(row[5])
            else:
                col_1.append(row[0])
                col_2.append(row[1])
                col_3.append(row[2])
                col_4.append(row[3])
                col_5.append(row[4])
    col_1.append("overall")
    col_2.append(precision_score(y_true, y_prediction))
    col_3.append(recall_score(y_true, y_prediction))
    col_4.append(f1_score(y_true, y_prediction))
    col_5.append(roc_auc_score(y_true, y_prediction))
    result = pd.DataFrame()
    result[columns[0]] = col_1
    result[columns[1]] = col_2
    result[columns[2]] = col_3
    result[columns[3]] = col_4
    result[columns[4]] = col_5
    print("——————Test——————")
    print(result)

In [ ]:
!pip install numpy==1.16.1
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
Y_train = np.load("Y_train.npy")
Y_texs = np.load("Y_test.npy")

ValueError: ignored

In [ ]:
X_train.shape

(217, 158)

In [ ]:
Xtrainnew = X_train[:148]

In [ ]:
Xtrainnew.shape

(148, 154)

In [ ]:
len(Y_train)

148

In [ ]:
X_train_T = X_train.transpose()

In [ ]:
X_train_T

array([[0.0, 2, 2, ..., 3, 3, 3],
       [0.0, 1430094402, 1430094402, ..., 1458049517, 1458049517,
        1458049517],
       [0.0, 'DE,EN,ES,ID,PT,RU,', 'DE,EN,ES,ID,PT,RU,', ...,
        'DE,EN,FR,RU,TR,', 'DE,EN,FR,RU,TR,', 'DE,EN,FR,RU,TR,'],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
X_test.shape

(158, 158)

In [ ]:
len(Y_test)

76

In [ ]:
X_test_new = X_test[:76]

In [ ]:
X_test_new.shape

(76, 154)

In [ ]:
Xtrainnew[1]

array([2, 1430094402, 33, 1, 0.16920418185882333, 0.16920418185882333,
       0.16920418185882333, 0.22560557581176444, 0.16920418185882333, 0.0,
       0.0, 0.22560557581176444, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.2820069697647055, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.2759480051946887, 0.0, 0.16920418185882333,
       0.16920418185882333, 0.2820069697647055, 0.16920418185882333,
       0.11280278790588222, 0.0, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.0, 0.11280278790588222, 0.11280278790588222,
       0.16920418185882333, 0.16920418185882333, 0.13797400259734435,
       0.13797400259734435, 0.11280278790588222, 0.11280278790588222, 0.0,
       0.0, 0.22560557581176444, 0.11280278790588222, 0.11280278790588222,
       0.16920418185882333, 0.20696100389601654, 0.13797400259734435,
       0.16920418185882333, 0.16920418185882333, 0.0, 0.05640139395294111,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       

In [ ]:
Xtrainnew = np.delete(Xtrainnew, 4, axis=1)

In [ ]:
X_test_new[1]

array([0, 1316040758, 39, 2, 0.041723918229578924, 0.12517175468873676,
       0.12517175468873676, 0.12517175468873676, 0.041723918229578924,
       0.12607512957032954, 0.12607512957032954, 0.2086195911478946,
       0.12517175468873676, 0.08344783645915785, 0.12607512957032954,
       0.2086195911478946, 0.08344783645915785, 0.08344783645915785, 0.0,
       0.15310302455421257, 0.12607512957032954, 0.1668956729183157,
       0.12517175468873676, 0.1668956729183157, 0.12517175468873676,
       0.37551526406621033, 0.2521502591406591, 0.08344783645915785,
       0.08344783645915785, 0.12607512957032954, 0.12607512957032954,
       0.29206742760705245, 0.08344783645915785, 0.12517175468873676,
       0.1668956729183157, 0.0, 0.0, 0.1668956729183157,
       0.08344783645915785, 0.12607512957032954, 0.2521502591406591,
       0.1668956729183157, 0.29206742760705245, 0.08344783645915785,
       0.08344783645915785, 0.0, 0.0, 0.08344783645915785,
       0.12517175468873676, 0.0, 0.04172391

In [ ]:
X_test_new = np.delete(X_test_new, 4, axis=1)

In [ ]:
new_xt.shape

(76, 144)

In [ ]:
len(Y_test)

76

In [ ]:
X_test

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0, 1316040758, 'AR,EN,', ..., 0.0, 0.0, 0.0],
       [0, 1316040758, 'AR,EN,', ..., 0.0, 0.6835954182319113, 0.0],
       ...,
       [1, 1429235843, 'EN,KI,SW,', ..., 0.0, 0.0, 0.0],
       [1, 1429235843, 'EN,KI,SW,', ..., 0.0, 0.0, 0.0],
       [1, 1429235843, 'EN,KI,SW,', ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
xt.shape

(158, 144)

In [ ]:
X_test.shape

(158, 158)

In [ ]:
xt = X_test

In [ ]:
xt = np.delete(xt, 4, axis=1)

In [ ]:
new_xt = xt[:76]

In [ ]:
array1 = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
array1[:3]

array([[1, 2],
       [3, 4],
       [5, 6]])

In [ ]:
xt[1]

array([0, 1316040758, 39, 2, 0.041723918229578924, 0.12517175468873676,
       0.12517175468873676, 0.12517175468873676, 0.041723918229578924,
       0.12607512957032954, 0.12607512957032954, 0.2086195911478946,
       0.12517175468873676, 0.08344783645915785, 0.12607512957032954,
       0.2086195911478946, 0.08344783645915785, 0.08344783645915785, 0.0,
       0.15310302455421257, 0.12607512957032954, 0.1668956729183157,
       0.12517175468873676, 0.1668956729183157, 0.12517175468873676,
       0.37551526406621033, 0.2521502591406591, 0.08344783645915785,
       0.08344783645915785, 0.12607512957032954, 0.12607512957032954,
       0.29206742760705245, 0.08344783645915785, 0.12517175468873676,
       0.1668956729183157, 0.0, 0.0, 0.1668956729183157,
       0.08344783645915785, 0.12607512957032954, 0.2521502591406591,
       0.1668956729183157, 0.29206742760705245, 0.08344783645915785,
       0.08344783645915785, 0.0, 0.0, 0.08344783645915785,
       0.12517175468873676, 0.0, 0.04172391

In [ ]:
Y.shape

(217, 148)

In [ ]:
X = Y.transpose()

In [ ]:
X.shape

(148, 217)

In [ ]:
len(Y_train)

148

In [ ]:
X_train

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [2, 1430094402, 'DE,EN,ES,ID,PT,RU,', ..., 0.0, 0.0, 0.0],
       [2, 1430094402, 'DE,EN,ES,ID,PT,RU,', ..., 0.0, 0.0, 0.0],
       ...,
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0],
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0],
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0]],
      dtype=object)

In [ ]:
sp = X_train

In [ ]:
sp

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [2, 1430094402, 'DE,EN,ES,ID,PT,RU,', ..., 0.0, 0.0, 0.0],
       [2, 1430094402, 'DE,EN,ES,ID,PT,RU,', ..., 0.0, 0.0, 0.0],
       ...,
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0],
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0],
       [3, 1458049517, 'DE,EN,FR,RU,TR,', ..., 0.0, 0.0, 0.0]],
      dtype=object)

In [ ]:
sp.shape

(217, 158)

In [ ]:
Y = np.delete(sp, 2, axis=1)

In [ ]:
Y = np.delete(Y, 2, axis=1)

In [ ]:
Y = np.delete(Y, 148, axis=1)

In [ ]:
Y[1]

array([2, 1430094402, 33, 1, 0.16920418185882333, 0.16920418185882333,
       0.16920418185882333, 0.22560557581176444, 0.16920418185882333, 0.0,
       0.0, 0.22560557581176444, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.2820069697647055, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.2759480051946887, 0.0, 0.16920418185882333,
       0.16920418185882333, 0.2820069697647055, 0.16920418185882333,
       0.11280278790588222, 0.0, 0.11280278790588222, 0.11280278790588222,
       0.0, 0.0, 0.11280278790588222, 0.11280278790588222,
       0.16920418185882333, 0.16920418185882333, 0.13797400259734435,
       0.13797400259734435, 0.11280278790588222, 0.11280278790588222, 0.0,
       0.0, 0.22560557581176444, 0.11280278790588222, 0.11280278790588222,
       0.16920418185882333, 0.20696100389601654, 0.13797400259734435,
       0.16920418185882333, 0.16920418185882333, 0.0, 0.05640139395294111,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       

In [ ]:
lr = LinearRegression()
lr.fit(Xtrainnew, Y_train)
y_pred = lr.predict(X_test_new)
y_pred = [0 if i<0.5 else 1 for i in y_pred]
show_result(Y_test, y_pred)

——————Test——————
          class precision    recall  f1-score   support
0             0    0.5067    1.0000    0.6726        38
1             1    1.0000    0.0263    0.0513        38
2      accuracy                        0.5132        76
3     macro avg    0.7533    0.5132    0.3619        76
4  weighted avg    0.7533    0.5132    0.3619        76
5       overall       1.0  0.026316  0.051282  0.513158


In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(Xtrainnew, Y_train)
y_pred = lr.predict(X_test_new)
show_result(Y_test, y_pred)

——————Test——————
          class precision    recall  f1-score   support
0             0    0.5067    1.0000    0.6726        38
1             1    1.0000    0.0263    0.0513        38
2      accuracy                        0.5132        76
3     macro avg    0.7533    0.5132    0.3619        76
4  weighted avg    0.7533    0.5132    0.3619        76
5       overall       1.0  0.026316  0.051282  0.513158


In [ ]:
nb = GaussianNB()
nb.fit(Xtrainnew, Y_train)
y_pred = nb.predict(X_test_new)
show_result(Y_test, y_pred)

——————Test——————
          class precision    recall  f1-score   support
0             0    0.6744    0.7632    0.7160        38
1             1    0.7273    0.6316    0.6761        38
2      accuracy                        0.6974        76
3     macro avg    0.7008    0.6974    0.6961        76
4  weighted avg    0.7008    0.6974    0.6961        76
5       overall  0.727273  0.631579  0.676056  0.697368


In [ ]:
dt = DecisionTreeClassifier()
dt.fit(Xtrainnew, Y_train)
y_pred = dt.predict(X_test_new)
show_result(Y_test, y_pred)

——————Test——————
          class precision    recall f1-score   support
0             0    0.5882    0.5263   0.5556        38
1             1    0.5714    0.6316   0.6000        38
2      accuracy                       0.5789        76
3     macro avg    0.5798    0.5789   0.5778        76
4  weighted avg    0.5798    0.5789   0.5778        76
5       overall  0.571429  0.631579      0.6  0.578947


In [ ]:
rf = RandomForestClassifier()
rf.fit(Xtrainnew, Y_train)
y_pred = rf.predict(X_test_new)
show_result(Y_test, y_pred)

——————Test——————
          class precision    recall  f1-score   support
0             0    0.4167    0.2632    0.3226        38
1             1    0.4615    0.6316    0.5333        38
2      accuracy                        0.4474        76
3     macro avg    0.4391    0.4474    0.4280        76
4  weighted avg    0.4391    0.4474    0.4280        76
5       overall  0.461538  0.631579  0.533333  0.447368


In [ ]:
Xtrainnew = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test_new= np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

TypeError: ignored

In [ ]:
np.save("X_train.npy",Xtrainnew[1:,])
np.save("Y_train.npy",np.array(Y_train))
np.save("X_test.npy",X_test_new[1:,])
np.save("Y_test.npy",np.array(Y_test))


In [ ]:
pd.DataFrame(Xtrainnew).to_csv("X_train.csv")
pd.DataFrame(Y_train).to_csv("Y_train.csv")
pd.DataFrame(X_test_new).to_csv("X_test.csv")
pd.DataFrame(Y_test).to_csv("Y_test.csv")